In [1]:
from datasets import load_dataset

In [2]:
# applying the first mask segmentation to the four pictures

# importing packages
import torch
import sys
import os
import pandas as pd

# running another code already done and saved
sys.path.append("./2018-NEON-beetles-processing/segmentation") 
from predict_masks import *


# the two layers of the image
classes = {0: 'background', 1: 'beetle'}
csv_annot = './data/BeetleMeasurements.csv'
result_path = './data/seg_results.csv' #this is already telling how many beetles there is in the image

# Leverage GPU if available
use_cuda = torch.cuda.is_available()
DEVICE   = torch.device("cuda:0" if use_cuda else "cpu")
print("Device: ", DEVICE)

if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

# read beetle msmt csv in -- load the csv file in the environment (the main dataset)
beetle_measurements = pd.read_csv(csv_annot)
# beetle_measurements = beetle_measurements.drop(columns=['Unnamed: 0'])

# read images in
dataset_folder = './data/input/*'
mask_folder = './data/mask/'
result_folder = './data/result/'
image_filepaths = read_image_paths(dataset_folder)
print(f'Number of images in dataset: {len(image_filepaths)}')

# establish df to store segmentation info
dataset_segmented = pd.DataFrame(columns = ['image', 'num_beetles'])

# load SAM model
segmentation_model = get_sam_model(DEVICE) #set of weight sthat you load into the model - someone else have already wrote it down
segmentation_model = SamPredictor(sam)


#preparing for a loop to apply the SAM model to all the images)
i = 0 #df indexer
print("Segmenting beetle images...")
for fp in image_filepaths:
    print(fp)

    #create a mask where beetle pixels are 1s and everything else is 0s
    mask, num_beetles = get_mask(fp, #image
                                 segmentation_model, #which model to appply
                                 beetle_measurements) 
    
    #create the path to which the mask will be saved
    mask_path = os.path.join(mask_folder, fp.split('/')[-1]) #replace extension and save mask as a png
    
    os.makedirs(mask_folder, exist_ok=True)
    
    #save mask with cv2 to preserve pixel categories
    print(f"Mask path:{mask_path}")
    cv2.imwrite(mask_path, mask)

    #enter relevant segmentation data for the image in our dataframe
    dataset_segmented.loc[i, 'image'] = fp.split('/')[-1]
    dataset_segmented.loc[i, 'num_beetles'] = num_beetles

    i += 1

# Save csv containing information about segmentation masks per each image
dataset_segmented.to_csv(result_path, index=False)




Device:  cuda:0
__CUDNN VERSION: 90100
__Number CUDA Devices: 1
__CUDA Device Name: GRID A100X-40C
__CUDA Device Total Memory [GB]: 42.945347584
Number of images in dataset: 0


NameError: name 'sam' is not defined

In [ ]:
# second step: take out the black background

from remove_background import *

dataset_images, image_filepaths = load_dataset_images(dataset_folder, 1)
main_folder_name = dataset_folder.split('/')[-1]

#load in our masks
mask_dataset_folder = mask_folder
dataset_masks, mask_filepaths = load_dataset_images(mask_folder)

errors = []
for (image, mask), fp in zip(zip(dataset_images, dataset_masks), image_filepaths): 
    #remove background from color image and replace it with a WHITE background
    img_removed_background = remove_background_color_based(image, mask) 

    #save the image with the removed background under its own folder
    bck_img_path = os.path.join(result_folder, fp.split('/')[-1])
    
    fn = "/" + bck_img_path.split('/')[-1]
    # background_folder = bck_img_path.replace(fn, "")
    os.makedirs(result_folder, exist_ok=True)

    #save the resized cropped wings to their path
    try:
        cv2.imwrite(bck_img_path, cv2.cvtColor(img_removed_background, cv2.COLOR_RGB2BGR))
    except FileNotFoundError:
        errors.append(bck_img_path)
        
print('The following images could encountered errors during background removal:', errors)
